In [11]:
# UT-TOR-DATA-PT-01-2020-U-C Week 12
# ETL Project
# Scraping https://www.rahmanmotors.com/cars/used/
# (c) Boris Smirnov

In [12]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import json
import time
import datetime as dt
import re

In [13]:
# Dealers description
dealer_dct = {
    'id': 'rahmanmotors',
    'name': 'Rahman Motors',
    'street': '6455 Mississauga Rd',
    'city': 'Mississauga',
    'zip': 'L5N1A6',
    'province': 'ON',
    'phone': '647-740-9312',
    'url': 'https://www.rahmanmotors.com',
    'latitude': '43.597561',
    'longitude': '-79.734300'
}

print(json.dumps(dealer_dct, indent=4))

with open('data/rahmanmotors.json', 'w', encoding='utf-8') as fp:
    json.dump(dealer_dct, fp, indent=4)

{
    "id": "rahmanmotors",
    "name": "Rahman Motors",
    "street": "6455 Mississauga Rd",
    "city": "Mississauga",
    "zip": "L5N1A6",
    "province": "ON",
    "phone": "647-740-9312",
    "url": "https://www.rahmanmotors.com",
    "latitude": "43.597561",
    "longitude": "-79.734300"
}


In [4]:
base_url = 'https://www.rahmanmotors.com'
inventory_url = base_url + '/used-cars/'

In [5]:
# Function uses inventory url (defined above), takes url parameters (such as a page number) as a dict,
# parses the inventory page and returns a list of URLs to car description pages
# Parsing stops when first sold vehicle found or when no search results found on a page
# The function returns a list or URLs and a boolean: True if there might be more data, False when done
def parse_inventory(query_args):
    global base_url, inventory_url
    res = []

    time.sleep(1.1)
    response = requests.get(inventory_url, query_args)
    
    soup = BeautifulSoup(response.text, 'lxml')
    vehicles = soup.find_all('div', class_='search-result-item')
    if vehicles is None:
        return res, False
    
    for v in vehicles:
        # check if it isn't sold
        sold = v.find('div', class_='SoldVehicle')
        if sold is not None:
            return res, False # all the rest of the inventory is a list of sold vehicles, no need to go further

        # get vehicle description page url
        url = base_url + v.div.contents[1].div.div.a['href']
        
        # Printing URL (for debugginh purposes)
        print(url)
        res.append(url)
    
    return res, True

In [6]:
# Making a list of inventory URLs
vehicle_urls = []
cpage = 1

urls, has_more = parse_inventory({'ppage': 100, 'cpage': cpage})

while has_more:
    vehicle_urls.extend(urls)
    cpage += 1
    urls, has_more = parse_inventory({'cpage': cpage})
    
vehicle_urls.extend(urls)
print(len(vehicle_urls))

https://www.rahmanmotors.com/cars/used/2008-Infiniti-EX35-SUV-108372
https://www.rahmanmotors.com/cars/used/2010-Volkswagen-Golf-Wagon-Wagon-106429
https://www.rahmanmotors.com/cars/used/2011-Hyundai-Sonata-Sedan-106426
https://www.rahmanmotors.com/cars/used/2016-Hyundai-Sonata-Sedan-104746
https://www.rahmanmotors.com/cars/used/2013-Ford-Focus-Sedan-104762
https://www.rahmanmotors.com/cars/used/2014-Ford-Fiesta--104758
https://www.rahmanmotors.com/cars/used/2012-Ford-Focus-Hatchback-104749
https://www.rahmanmotors.com/cars/used/2007-GMC-Acadia-SUV-104759
https://www.rahmanmotors.com/cars/used/2005-Nissan-Altima-Sedan-104752
https://www.rahmanmotors.com/cars/used/2015-Subaru-Impreza-Hatchback-104767
https://www.rahmanmotors.com/cars/used/2006-Volvo-S80-Sedan-104748
https://www.rahmanmotors.com/cars/used/2011-Nissan-Altima-Sedan-104751
https://www.rahmanmotors.com/cars/used/2006-Volkswagen-Jetta-Wagon-Wagon-104753
https://www.rahmanmotors.com/cars/used/2010-Volvo-XC90-SUV-104750
https:/

In [9]:
# Function parses vehicle description page and returns a dictionary with its parameters
def parse_car_infopage(url):
    # Translation table from their names to our key names
    translation_table = {
        'Make:': 'make',
        'Model:': 'model',
        'Year:': 'year',
        'Body Style:': 'body_style',
        'Odometer:': 'mileage', # Note: special treatment
        'Transmission:': 'transmission',
        'Engine:': 'cylinders', # Note: special treatment
        'Engine Size:': 'displacement',
        'Driveline:': 'drivetrain',
        'Exterior Color:': 'ext_color', 
        'Interior Color:': 'int_color',
        'Doors:': 'doors', # Note: special treatment
        'Passengers:': 'passengers',
        'Stock Number:': 'stock_id',
        'VIN:': 'vin'
    }
    
    # Fetching the page
    time.sleep(1.1)
    response = requests.get(url)

    # Parsing
    soup = BeautifulSoup(response.text, 'lxml')
    price_node = soup.find('span', class_='PriceValue')
    details_node = soup.find('div', class_='VehicleInfoDetails')
    descr_node = soup.find('div', class_='seller_comments')

    # Basic info
    res = {
        'timestamp': dt.datetime.now().isoformat(),
        'dealer': 'rahmanmotors',
        'url': url
    }

    # Price
    price = re.sub('[^0-9]', '', price_node.contents[0]) # Cleaning junk, leaving only numbers
    res['price'] = price

    # Reading core parameters
    spans = details_node.find_all('span')

    for s in spans:
        if s.text in translation_table:
            key = translation_table[s.text]
            value = s.next_sibling.text

            # Ad Hoc treatment
            if key in ['cylinders', 'doors']:
                value = value.split()[0]
            elif key == 'mileage':
                value = re.sub('[^0-9]', '', value) # Remove all non numbers using regexp substitution

            res[key] = value
    
    # Free text description        
    res['description'] = descr.find('p').text

    # Printing result (for debugging purposes)
    print(json.dumps(res, indent=4))
    
    # Done
    return res

In [10]:
# Now scraping vehicle data, one by one
vehicle_data = []

for url in vehicle_urls:
    vehicle_data.append(parse_car_infopage(url))
    
print(len(vehicle_data))

with open('data/rahmanmotors_cars.json', 'w', encoding='utf-8') as fp:
    json.dump(vehicle_data, fp, indent=4)

{
    "timestamp": "2020-04-24T14:30:24.256747",
    "dealer": "rahmanmotors",
    "url": "https://www.rahmanmotors.com/cars/used/2008-Infiniti-EX35-SUV-108372",
    "price": "6500",
    "make": "Infiniti",
    "model": "EX35",
    "year": "2008",
    "body_style": "SUV",
    "mileage": "206000",
    "transmission": "Automatic",
    "cylinders": "6",
    "displacement": "3.5",
    "drivetrain": "AWD",
    "ext_color": "Grey",
    "int_color": "Black",
    "doors": "4",
    "passengers": "5",
    "stock_id": "788",
    "vin": "JNKAJ09F58M355662",
    "description": "2008 INFINITI EX35, ALL WHEEL DRIVE (AWD), 206K, AUTOMATIC, 4 DOOR, FULLY LOADED, PUSH-BUTTON START, LEATHER INTERIOR, POWER WINDOWS, POWER LOCKS, POWER SEATS, HEATED SEATS, SUNROOF, KEY-LESS ENTRY, BLUETOOTH, CD PLAYER, AUX, RADIO, A/C, ALLOY WHEELS, CLEAN CAR-FAX REPORT (WILL PROVIDE CAR-FAX REPORT), ONTARIO VEHICLE, EXCELLENT CONDITION, FULLY CERTIFIED."
}
{
    "timestamp": "2020-04-24T14:30:26.050957",
    "dealer": "ra